In [9]:
import cv2
import numpy as np

In [ ]:
class SITDisplay:
    COLORS = {
        'red': (0,0,255), 'green': (127,255,0), 'blue': (255,127,0), 'yellow': (0,127,255), \
        'black': (0,0,0), 'white': (255,255,255), 'pink': (255, 0, 255), 'tortoise': (0, 255, 255)
    }

    island_colors = {2: 'green', 3: 'blue', 4: 'yellow', 5: 'black'}
    animal_colors = {1: 'tortoise', 2: 'pink', 3: 'white'}

    def __init__(self, pt, cfg):  # injected dependency - position controller
        self.pt = pt
        self.cfg = cfg
        self.masked_frame = None
        
    def render(self, input_frame, status, islands=[], angles=[], text_infos=[]):
        COLORS = SITDisplay.COLORS
        FONT = cv2.FONT_HERSHEY_DUPLEX
        status_color = COLORS['green'] if status.value == 1 else COLORS['red']
        texts = list(text_infos)
        
        # mask unused areas
        frame = cv2.bitwise_and(src1=input_frame, src2=self.pt.mask)
        self.masked_frame = frame.copy()

        # size of the arena and status indicator
        cv2.circle(frame, (self.pt.cfg['arena_x'], self.pt.cfg['arena_y']), self.pt.cfg['arena_radius'], COLORS['red'], 2)
        cv2.circle(frame, (self.pt.cfg['arena_x'], self.pt.cfg['arena_y']), self.pt.cfg['floor_radius'], COLORS['red'], 2)
        cv2.circle(frame, (20,20), 10, status_color, -6)
        
        # draw islands
        if len(islands) > 0:
            for island in islands:
                clr = COLORS[SITDisplay.island_colors[island.sound_id]]
                x, y = self.pt.meters_to_px(island.x, island.y)
                cv2.circle(frame, (x, y), int(island.r/self.pt.pixel_size), clr, 2)
            
        # draw anglular indicators
        if len(angles) > 0:  # should not be more than 4
            p_rho = self.pt.cfg['floor_radius'] * self.pt.pixel_size
            for p_phi, clr in zip(angles, ['blue', 'yellow', 'pink', 'tortoise'][:len(angles)]):
                x_init, y_init = self.pt.meters_to_px(p_rho * np.sin(p_phi), p_rho * np.cos(p_phi))
                cv2.circle(frame, (x_init, y_init), 10, COLORS[clr], -1)

        # light or dark period
        texts.append('Light' if self.pt.is_light else 'Dark')

        if not self.cfg['save_contours']:
            frame_to_save = frame.copy()  # video without animal contours
        
        # draw animal position center(s) and contours
        if self.pt.positions_in_px is not None:
            pos_in_px = self.pt.positions_in_px
            pos_in_m = self.pt.positions_in_m
            contours = self.pt.contours
            
            for i in range(len(pos_in_px)):
                clr = COLORS[SITDisplay.animal_colors[i+1]]
                x_px, y_px, alpha = pos_in_px[i][0], pos_in_px[i][1], self.pt.hds[0]
                
                # draw animal center and contours
                cv2.circle(frame, (x_px, y_px), 5, clr, -1)
                cv2.drawContours(frame, [contours[i]], 0, clr, 1, cv2.LINE_AA)
                
                # draw (instant position values and) HD vector
                #cv2.putText(frame, '%.2f %.2f' % (pos_in_m[i][0], pos_in_m[i][1]), (x_px + 10, y_px + 10), FONT, .3, clr)
                #x1, y1 = int(x_px + 40*np.cos(np.deg2rad(alpha))), int(y_px - 40*np.sin(np.deg2rad(alpha)))
                #cv2.line(frame, (x_px, y_px), (x1, y1), clr, 1)
                
                texts.append('Animal %d: %.3f %.3f' % (i+1, pos_in_m[i][0], pos_in_m[i][1]))
                texts.append('Speed/HD: %.2f %.2f' % (self.pt.speeds[0], self.pt.hds[0]))

        if self.cfg['save_contours']:
            frame_to_save = frame.copy()  # save video with animal contours
            
        # draw text infos
        for i, text in enumerate(texts):
            cv2.putText(frame, text, (10, 30 + 20*(i+1)), FONT, .5, COLORS['white'])
            cv2.putText(frame_to_save, text, (10, 30 + 20*(i+1)), FONT, .5, COLORS['white'])

        return frame, frame_to_save